# Episode 8 - Let’s Write a Decision Tree Classifier from Scratch

我们将用纯Python编写一个决策树分类器。 在每种方法下面，我写了一个小示例来帮助解释它的功能。

In [1]:
# For Python 2 / 3 compatability
from __future__ import print_function

In [2]:
# 样例数据集。
# 格式：每一行都是一个例子。
# 最后一列是标签。
# 前两列是功能。
# 通过添加更多功能和示例，随意使用它。
# 有趣的说明：第二和第五个例子，有相同的功能，但标签不同
# 决策树处理这种情况。

training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

In [3]:
# 列标签。
# 这些仅用于打印树。
header = ["color", "diameter", "label"]

In [4]:
def unique_vals(rows, col):
    # 数据集中第col列所出现的可能值。
    return set([row[col] for row in rows])

In [5]:
#######
# Demo:
unique_vals(training_data, 0)
# unique_vals(training_data, 1)
#######

{'Green', 'Red', 'Yellow'}

In [6]:
def class_counts(rows):
    """计算训练集中每种标签数据的数量"""
    counts = {}  # a dictionary of label -> count.
    for row in rows:
        # 在我们的数据集格式中，标签始终是最后一列
        label = row[-1]
        if label not in counts:
            counts[label] = 0
        counts[label] += 1
    return counts

In [7]:
#######
# Demo:
class_counts(training_data)
#######

{'Apple': 2, 'Grape': 2, 'Lemon': 1}

In [8]:
def is_numeric(value):
    # 测试一个值是否是数字。
    return isinstance(value, int) or isinstance(value, float)

In [9]:
#######
# Demo:
is_numeric(7)
# is_numeric("Red")
#######

True

In [10]:
class Question:
    """问题用于分割数据集。

     这个类只记录一个“列号”（例如，颜色为0）和a
     '列值'（例如，绿色）。 'match'方法用于比较
     在一个例子中的特征值到存储在特征值中的特征值
     题。 看下面的演示。
    """

    def __init__(self, column, value):
        self.column = column
        self.value = value

    def match(self, example):
        # 将示例中的特征值与这个问题中的特征值进行比较。
        val = example[self.column]
        if is_numeric(val):
            return val >= self.value
        else:
            return val == self.value

    def __repr__(self):
        # 这只是一个帮助打印可读格式的问题的方法。
        condition = "=="
        if is_numeric(self.value):
            condition = ">="
        return "Is %s %s %s?" % (
            header[self.column], condition, str(self.value))

In [11]:
#######
# Demo:
# 我们来写一个数字属性的问题
Question(1, 3)

Is diameter >= 3?

In [12]:
# 如何为一个分类属性
q = Question(0, 'Green')
q

Is color == Green?

In [13]:
# 从训练集中选择一个例子
example = training_data[0]
# 观察是否与问题匹配
q.match(example) # 应返回true，因为第一个例子是绿色。
#######

True

In [14]:
def partition(rows, question):
    """分区数据集。

     对于数据集中的每一行，检查它是否与问题匹配。 如果是，将它添加到'真行'，否则，将它添加到'假行'。
    """
    true_rows, false_rows = [], []
    for row in rows:
        if question.match(row):
            true_rows.append(row)
        else:
            false_rows.append(row)
    return true_rows, false_rows

In [15]:
#######
# Demo:
# 我们根据行是否是红色来划分训练数据。
true_rows, false_rows = partition(training_data, Question(0, 'Red'))
# 这将包含所有'红'行。
true_rows

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]

In [16]:
# 这将包含其他所有东西。
false_rows
#######

[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]

In [17]:
def gini(rows):
    """计算行列表的基尼杂质。

     有几种不同的方式来做到这一点，我认为这是一个
     最简洁。 看到：
    https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    counts = class_counts(rows)
    impurity = 1
    for lbl in counts:
        prob_of_lbl = counts[lbl] / float(len(rows))
        impurity -= prob_of_lbl**2
    return impurity

In [18]:
#######
# Demo:
# 让我们看看一些例子来了解基尼杂质是如何工作的。
#
# 首先，我们将看看没有混合的数据集。
no_mixing = [['Apple'],
              ['Apple']]
# this will return 0
gini(no_mixing)

0.0

In [19]:
# 现在，我们将以苹果：橙子-50:50比例来看待数据集
some_mixing = [['Apple'],
               ['Orange']]
# 这将返回0.5 - 这意味着，有50％的错误分类机会
# 我们从数据集中绘制的随机示例。
gini(some_mixing)

0.5

In [20]:
# 现在，我们将查看包含许多不同标签的数据集
lots_of_mixing = [['Apple'],
                  ['Orange'],
                  ['Grape'],
                  ['Grapefruit'],
                  ['Blueberry']]
# This will return 0.8
gini(lots_of_mixing)
#######

0.7999999999999998

In [21]:
def info_gain(left, right, current_uncertainty):
    """信息收益。

     起始节点的不确定性，减去加权杂质
     两个子节点。
    """
    p = float(len(left)) / (len(left) + len(right))
    return current_uncertainty - p * gini(left) - (1 - p) * gini(right)

In [22]:
#######
# Demo:
# 计算我们训练数据的不确定性。
current_uncertainty = gini(training_data)
current_uncertainty

0.6399999999999999

In [23]:
# 通过划分“绿色”我们获得了多少信息？
true_rows, false_rows = partition(training_data, Question(0, 'Green'))
info_gain(true_rows, false_rows, current_uncertainty)

0.1399999999999999

In [24]:
# 如果我们用红色划分会怎么样呢
true_rows, false_rows = partition(training_data, Question(0,'Red'))
info_gain(true_rows, false_rows, current_uncertainty)

0.37333333333333324

In [25]:
# 看起来我们从‘红色’ (0.37)中相比 '绿色' (0.14)学习更多。
# Why? Look at the different splits that result, and see which one
# looks more 'unmixed' to you.
true_rows, false_rows = partition(training_data, Question(0,'Red'))

# 在这里，true_rows只包含'葡萄'。
true_rows

[['Red', 1, 'Grape'], ['Red', 1, 'Grape']]

In [26]:
# And the false rows contain two types of fruit. Not too bad.
false_rows

[['Green', 3, 'Apple'], ['Yellow', 3, 'Apple'], ['Yellow', 3, 'Lemon']]

In [27]:
# 另一方面，Green的分区并没有太大的帮助。
true_rows, false_rows = partition(training_data, Question(0,'Green'))

# 我们在真行中隔离了一个苹果。
true_rows

[['Green', 3, 'Apple']]

In [28]:
# But, the false-rows are badly mixed up.
false_rows
#######

[['Yellow', 3, 'Apple'],
 ['Red', 1, 'Grape'],
 ['Red', 1, 'Grape'],
 ['Yellow', 3, 'Lemon']]

In [29]:
def find_best_split(rows):
    """通过迭代每个特征/值来找出最佳问题
     并计算信息增益。"""
    best_gain = 0  # keep track of the best information gain
    best_question = None  # keep train of the feature / value that produced it
    current_uncertainty = gini(rows)
    n_features = len(rows[0]) - 1  # number of columns

    for col in range(n_features):  # for each feature

        values = set([row[col] for row in rows])  # unique values in the column

        for val in values:  # for each value

            question = Question(col, val)

            # try splitting the dataset
            true_rows, false_rows = partition(rows, question)

            # Skip this split if it doesn't divide the
            # dataset.
            if len(true_rows) == 0 or len(false_rows) == 0:
                continue

            # Calculate the information gain from this split
            gain = info_gain(true_rows, false_rows, current_uncertainty)

            # You actually can use '>' instead of '>=' here
            # but I wanted the tree to look a certain way for our
            # toy dataset.
            if gain >= best_gain:
                best_gain, best_question = gain, question

    return best_gain, best_question

In [30]:
#######
# Demo:
# Find the best question to ask first for our toy dataset.
best_gain, best_question = find_best_split(training_data)
best_question
# FYI: is color == Red is just as good. See the note in the code above
# where I used '>='.
#######

Is diameter >= 3?

In [31]:
class Leaf:
    """叶节点分类数据。

     这包含一个类的词典（例如，“苹果”） - >次数
     它出现在来自训练数据的行中。
    """

    def __init__(self, rows):
        self.predictions = class_counts(rows)

In [32]:
class Decision_Node:
    """决策节点提出问题。

     这涉及到这个问题，以及两个子节点。
    """

    def __init__(self,
                 question,
                 true_branch,
                 false_branch):
        self.question = question
        self.true_branch = true_branch
        self.false_branch = false_branch

In [33]:
def build_tree(rows):
    """构建树。

     递归规则：1）相信它的工作原理。 2）开始检查
     对于基本情况（没有进一步的信息获得）。 3）准备
     巨大的堆栈痕迹。
    """

    # 尝试在每个独特属性上分配数据集，计算信息增益，并返回产生最高收益的问题。
    gain, question = find_best_split(rows)

    # Base case: no further info gain
    # Since we can ask no further questions,
    # we'll return a leaf.
    if gain == 0:
        return Leaf(rows)

    # If we reach here, we have found a useful feature / value
    # to partition on.
    true_rows, false_rows = partition(rows, question)

    # Recursively build the true branch.
    true_branch = build_tree(true_rows)

    # Recursively build the false branch.
    false_branch = build_tree(false_rows)

    # Return a Question node.
    # This records the best feature / value to ask at this point,
    # as well as the branches to follow
    # dependingo on the answer.
    return Decision_Node(question, true_branch, false_branch)

In [34]:
def print_tree(node, spacing=""):
    """World's most elegant tree printing function."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        print (spacing + "Predict", node.predictions)
        return

    # Print the question at this node
    print (spacing + str(node.question))

    # Call this function recursively on the true branch
    print (spacing + '--> True:')
    print_tree(node.true_branch, spacing + "  ")

    # Call this function recursively on the false branch
    print (spacing + '--> False:')
    print_tree(node.false_branch, spacing + "  ")

In [35]:
my_tree = build_tree(training_data)

In [36]:
print_tree(my_tree)

Is diameter >= 3?
--> True:
  Is color == Yellow?
  --> True:
    Predict {'Apple': 1, 'Lemon': 1}
  --> False:
    Predict {'Apple': 1}
--> False:
  Predict {'Grape': 2}


In [37]:
def classify(row, node):
    """See the 'rules of recursion' above."""

    # Base case: we've reached a leaf
    if isinstance(node, Leaf):
        return node.predictions

    # Decide whether to follow the true-branch or the false-branch.
    # Compare the feature / value stored in the node,
    # to the example we're considering.
    if node.question.match(row):
        return classify(row, node.true_branch)
    else:
        return classify(row, node.false_branch)

In [38]:
#######
# Demo:
# The tree predicts the 1st row of our
# training data is an apple with confidence 1.
classify(training_data[0], my_tree)
#######

{'Apple': 1}

In [39]:
def print_leaf(counts):
    """A nicer way to print the predictions at a leaf."""
    total = sum(counts.values()) * 1.0
    probs = {}
    for lbl in counts.keys():
        probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
    return probs

In [40]:
#######
# Demo:
# Printing that a bit nicer
print_leaf(classify(training_data[0], my_tree))
#######

{'Apple': '100%'}

In [41]:
#######
# Demo:
# On the second example, the confidence is lower
print_leaf(classify(training_data[1], my_tree))
#######

{'Apple': '50%', 'Lemon': '50%'}

In [42]:
# Evaluate
testing_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 4, 'Apple'],
    ['Red', 2, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]

In [43]:
for row in testing_data:
    print ("Actual: %s. Predicted: %s" %
           (row[-1], print_leaf(classify(row, my_tree))))

Actual: Apple. Predicted: {'Apple': '100%'}
Actual: Apple. Predicted: {'Apple': '50%', 'Lemon': '50%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Grape. Predicted: {'Grape': '100%'}
Actual: Lemon. Predicted: {'Apple': '50%', 'Lemon': '50%'}
